In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd

from utils_data import import_process_compas
from utils import discretize
pd.set_option('display.max_colwidth', None)
from utils import printable


In [ ]:
df = import_process_compas(input_dir="./data")

/home/epastor/projects/divexplorer-journal/divexplorer-general/utils_data.py:204: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["length_of_stay"] = pd.to_timedelta(


In [4]:
attributes = list(df.columns)
attributes.remove('class')
attributes.remove('predicted')

df_discretized = discretize(df, dataset_name='compas', attributes=attributes)  

In [5]:
abbreviations = {"age_cat":"age", "priors_count":"#prior", "Greater than 45":">45", "25 - 45":"25-45",\
                "African-American":"African-American", "c_charge_degree":"charge", "Less than 25":"<25", "=>":">", "=<":"<",\
                "length_of_stay":"stay", "Caucasian":"Caucasian", 'race': 'ethnicity'} 

# Divergence analysis

In [ ]:
min_support = 0.0175
min_support_count = min_support*len(df_discretized)
min_support_count

108.01

In [7]:
import resource

def limit_memory(max_mem_mb):
    """Limit memory usage of the current notebook process."""
    soft, hard = max_mem_mb * 1024 * 1024, max_mem_mb * 1024 * 1024
    resource.setrlimit(resource.RLIMIT_AS, (soft, hard))

# Example: Limit the notebook to 2 GB of memory
limit_memory(10*1024)

## True negative rate

In [8]:
target_col = 'true_negative'

df_discretized_analyze = df_discretized.copy()
df_discretized_analyze[target_col] = df_discretized_analyze['class'].apply(lambda x: 1 if x == 0 else 0)

df_discretized_analyze.drop(columns=["class", "predicted"], inplace=True)

type_outcome = 'boolean'

In [9]:
from divexplorer import DivergenceExplorer

fp_diver = DivergenceExplorer(df_discretized_analyze)

In [ ]:
import time
try:
    start_time = time.time()
    subgroups = fp_diver.get_pattern_divergence(min_support=min_support, boolean_outcomes=[target_col])
    print("--- %s seconds ---" % (time.time() - start_time))
except MemoryError:
    print("Memory limit exceeded!")

--- 0.08154964447021484 seconds ---


In [11]:
cols = ['itemset', 'support', target_col, f"{target_col}_div", f"{target_col}_t"]

In [12]:
from divexplorer import DivergencePatternProcessor


fp_details = DivergencePatternProcessor(subgroups, target_col)

In [13]:
import time
try:
    
    start_time = time.time()
    patterns = fp_details.get_patterns(th_redundancy=0)
    print("--- %s seconds ---" % (time.time() - start_time))
except MemoryError:
    print("Memory limit exceeded!")

--- 0.0019483566284179688 seconds ---


In [14]:
subgroups_print = patterns[cols].sort_values(by=f"{target_col}_div", ascending=False).head(3).copy()
subgroups_print = printable(subgroups_print, target_col, abbreviations=abbreviations, round_v=3)
display(subgroups_print)

print(subgroups_print.to_latex(index=False))

,itemset,support,true_negative,true_negative_div,true_negative_t
556,"age>45, stay<week, #prior=0, ethnicity=Caucasian, sex=Male",0.026,0.892,0.348,13.3
424,"age>45, stay<week, #prior=0, ethnicity=Caucasian",0.036,0.884,0.339,15.0
579,"age>45, charge=M, stay<week, #prior=0, ethnicity=Caucasian",0.024,0.872,0.328,11.4


\begin{tabular}{lrrrr}
\toprule
itemset & support & true_negative & true_negative_div & true_negative_t \\
\midrule
age>45, stay<week, #prior=0, ethnicity=Caucasian, sex=Male & 0.026000 & 0.892000 & 0.348000 & 13.300000 \\
age>45, stay<week, #prior=0, ethnicity=Caucasian & 0.036000 & 0.884000 & 0.339000 & 15.000000 \\
age>45, charge=M, stay<week, #prior=0, ethnicity=Caucasian & 0.024000 & 0.872000 & 0.328000 & 11.400000 \\
\bottomrule
\end{tabular}



In [15]:
patterns.loc[patterns['itemset']==frozenset()].iloc[0][target_col]

np.float64(0.5448801036941024)

## Predicted negative rate

In [16]:
from divexplorer.outcomes import get_false_positive_rate_outcome

target_col = 'pred_negative'

df_discretized_analyze = df_discretized.copy()
df_discretized_analyze[target_col] = df_discretized_analyze['predicted'].apply(lambda x: 1 if x == 0 else 0)

df_discretized_analyze.drop(columns=["class", "predicted"], inplace=True)

type_outcome = 'boolean'

In [17]:
from divexplorer import DivergenceExplorer

fp_diver = DivergenceExplorer(df_discretized_analyze)

In [18]:
import time
try:
    start_time = time.time()
    subgroups = fp_diver.get_pattern_divergence(min_support=min_support, boolean_outcomes=[target_col])
    print("--- %s seconds ---" % (time.time() - start_time))
except MemoryError:
    print("Memory limit exceeded!")

--- 0.09000921249389648 seconds ---


In [19]:
cols = ['itemset', 'support', target_col, f"{target_col}_div", f"{target_col}_t"]

In [20]:
from divexplorer import DivergencePatternProcessor


fp_details = DivergencePatternProcessor(subgroups, target_col)

In [21]:
import time
try:
    
    start_time = time.time()
    patterns = fp_details.get_patterns(th_redundancy=0)
    print("--- %s seconds ---" % (time.time() - start_time))
except MemoryError:
    print("Memory limit exceeded!")

--- 0.0021088123321533203 seconds ---


In [22]:
subgroups_print = patterns[cols].sort_values(by=[f"{target_col}_div", f'{target_col}_t'], ascending=[False, False]).head(3).copy()
subgroups_print = printable(subgroups_print, target_col, abbreviations=abbreviations, round_v=3)

display(subgroups_print)

print(subgroups_print.to_latex(index=False))

,itemset,support,pred_negative,pred_negative_div,pred_negative_t
379,"age>45, charge=M, #prior=0",0.042,1.000,0.185,29.0
556,"age>45, stay<week, #prior=0, ethnicity=Caucasian, sex=Male",0.026,1.000,0.185,22.6
322,"age>45, stay<week, #prior=0, sex=Male",0.049,0.997,0.182,26.4


\begin{tabular}{lrrrr}
\toprule
itemset & support & pred_negative & pred_negative_div & pred_negative_t \\
\midrule
age>45, charge=M, #prior=0 & 0.042000 & 1.000000 & 0.185000 & 29.000000 \\
age>45, stay<week, #prior=0, ethnicity=Caucasian, sex=Male & 0.026000 & 1.000000 & 0.185000 & 22.600000 \\
age>45, stay<week, #prior=0, sex=Male & 0.049000 & 0.997000 & 0.182000 & 26.400000 \\
\bottomrule
\end{tabular}



In [23]:
patterns.loc[patterns['itemset']==frozenset()].iloc[0][target_col]

np.float64(0.814646791963707)

## FPR

In [24]:
from divexplorer.outcomes import get_false_positive_rate_outcome

target_col = 'fp'

df_discretized_analyze = df_discretized.copy()

df_discretized_analyze[target_col] =  get_false_positive_rate_outcome(df_discretized_analyze['class'].values, df_discretized_analyze['predicted'].values)

df_discretized_analyze.drop(columns=["class", "predicted"], inplace=True)

type_outcome = 'boolean'

In [25]:
from divexplorer import DivergenceExplorer

fp_diver = DivergenceExplorer(df_discretized_analyze)

In [26]:
import time
try:
    start_time = time.time()
    subgroups = fp_diver.get_pattern_divergence(min_support=min_support, boolean_outcomes=[target_col])
    print("--- %s seconds ---" % (time.time() - start_time))
except MemoryError:
    print("Memory limit exceeded!")

--- 0.08045744895935059 seconds ---


In [27]:
cols = ['itemset', 'support', target_col, f"{target_col}_div", f"{target_col}_t"]

In [28]:
from divexplorer import DivergencePatternProcessor


fp_details = DivergencePatternProcessor(subgroups, target_col)

In [29]:
import time
try:
    
    start_time = time.time()
    patterns = fp_details.get_patterns(th_redundancy=None)
    print("--- %s seconds ---" % (time.time() - start_time))
except MemoryError:
    print("Memory limit exceeded!")

--- 0.0004863739013671875 seconds ---


In [30]:
subgroups_print = patterns[cols].sort_values(by=f"{target_col}_div", ascending=False).head(3).copy()
subgroups_print = printable(subgroups_print, target_col, abbreviations=abbreviations)

display(subgroups_print)

print(subgroups_print.to_latex(index=False))

,itemset,support,fp,fp_div,fp_t
656,"age<25, #prior>3, sex=Male",0.019,0.68,0.59,6.1
617,"age<25, #prior>3",0.022,0.62,0.53,5.7
655,"age<25, stay=1w-3M, ethnicity=African-American, sex=Male",0.019,0.39,0.31,3.8


\begin{tabular}{lrrrr}
\toprule
itemset & support & fp & fp_div & fp_t \\
\midrule
age<25, #prior>3, sex=Male & 0.019000 & 0.680000 & 0.590000 & 6.100000 \\
age<25, #prior>3 & 0.022000 & 0.620000 & 0.530000 & 5.700000 \\
age<25, stay=1w-3M, ethnicity=African-American, sex=Male & 0.019000 & 0.390000 & 0.310000 & 3.800000 \\
\bottomrule
\end{tabular}



In [31]:
patterns.loc[patterns['itemset']==frozenset()].iloc[0][target_col]

np.float64(0.08831400535236396)

## FNR

In [32]:
from divexplorer.outcomes import get_false_negative_rate_outcome

target_col = 'fn'

df_discretized_analyze = df_discretized.copy()

df_discretized_analyze[target_col] =  get_false_negative_rate_outcome(df_discretized_analyze['class'].values, df_discretized_analyze['predicted'].values)

df_discretized_analyze.drop(columns=["class", "predicted"], inplace=True)

type_outcome = 'boolean'

In [33]:
from divexplorer import DivergenceExplorer

fp_diver = DivergenceExplorer(df_discretized_analyze)

In [34]:
import time
try:
    start_time = time.time()
    subgroups = fp_diver.get_pattern_divergence(min_support=min_support, boolean_outcomes=[target_col])
    print("--- %s seconds ---" % (time.time() - start_time))
except MemoryError:
    print("Memory limit exceeded!")

--- 0.09435105323791504 seconds ---


In [35]:
cols = ['itemset', 'support', target_col, f"{target_col}_div", f"{target_col}_t"]

In [36]:
from divexplorer import DivergencePatternProcessor


fp_details = DivergencePatternProcessor(subgroups, target_col)

In [37]:
import time
try:
    
    start_time = time.time()
    patterns = fp_details.get_patterns(th_redundancy=0)
    print("--- %s seconds ---" % (time.time() - start_time))
except MemoryError:
    print("Memory limit exceeded!")

--- 0.002046823501586914 seconds ---


In [38]:
subgroups_print = patterns[cols].sort_values(by=[f"{target_col}_div", f"{target_col}_t"], ascending=[False, False]).head(3).copy()
subgroups_print = printable(subgroups_print, target_col, abbreviations=abbreviations)

display(subgroups_print)

print(subgroups_print.to_latex(index=False))

,itemset,support,fn,fn_div,fn_t
321,"age>45, charge=M, ethnicity=Caucasian",0.049,1.0,0.3,17.6
501,"age>45, charge=M, #prior=[1,3]",0.029,1.0,0.3,14.1
379,"age>45, charge=M, #prior=0",0.042,1.0,0.3,10.4


\begin{tabular}{lrrrr}
\toprule
itemset & support & fn & fn_div & fn_t \\
\midrule
age>45, charge=M, ethnicity=Caucasian & 0.049000 & 1.000000 & 0.300000 & 17.600000 \\
age>45, charge=M, #prior=[1,3] & 0.029000 & 1.000000 & 0.300000 & 14.100000 \\
age>45, charge=M, #prior=0 & 0.042000 & 1.000000 & 0.300000 & 10.400000 \\
\bottomrule
\end{tabular}



In [39]:
patterns.loc[patterns['itemset']==frozenset()].iloc[0][target_col]

np.float64(0.6984692061231755)